In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
import math
import word_dist_script as ws
import pandas as pd

In [2]:
twitEmbs = ws.load_twitter_embs()

loading finished


In [3]:
load_word2vec = True

if load_word2vec:
    word2vec = ws.load_word2vec_embs()


Loading word2vec model...
This can take several minutes...


In [ ]:
# every_emb = ws.get_every_embeddings(word2vec)
# clf = KMeans(n_clusters=5, random_state=0)

# clf_kmoids = KMeans(n_clusters=5, random_state=0, metric='precomputed', method='pam')

# clf_kmoids.fit(every_emb)

# pred_emb = clf_kmoids.predict(every_emb)

In [ ]:
every_dist = ws.get_every_distance(word2vec)
import random
random.seed(4012)

clf_kmoids = KMedoids(n_clusters=5, random_state=4012, metric='precomputed', method='pam')


clf_kmoids.fit(every_dist)

pred_dist = clf_kmoids.predict(every_dist)

In [ ]:
pred_dist

array([1, 0, 3, 4, 1, 0, 1, 3, 1, 2, 2, 2, 3, 1, 3, 0, 0, 1, 0, 1, 0, 0,
       1, 2, 0, 2, 2, 3, 0, 3, 4, 4, 0, 4, 1, 4, 2], dtype=int64)

In [ ]:
sorted_dist = ws.sort_zip_labels(pred_dist)
groups_dist = ws.group_by_cluster(sorted_dist)

# sorted_emb = ws.sort_zip_labels(pred_emb)
# groups_emb = ws.group_by_cluster(sorted_emb)

In [ ]:
for i, group in enumerate(groups_dist):
    print(i, group)

0 ['album', 'band', 'genre', 'location', 'metrics', 'artist', 'instrument', 'poem', 'java', 'song']
1 ['journal', 'award', 'book', 'conference', 'event', 'magazine', 'misc', 'Organisation', 'theory']
2 ['country', 'discipline', 'election', 'person', 'politics', 'politician', 'writer']
3 ['algorithm', 'chemical', 'enzyme', 'field', 'product', 'protein']
4 ['astronomer', 'researcher', 'scientist', 'task', 'university']


In [ ]:
# for i, group in enumerate(groups_emb):
#     print(i, group)

NameError: name 'groups_emb' is not defined

In [ ]:
for group in groups_dist:
    ## find distance to centroid
    label = ws.find_label_for_cluster(word2vec, group)
    print(label, group)

genre ['album', 'band', 'genre', 'location', 'metrics', 'artist', 'instrument', 'poem', 'java', 'song']
magazine ['journal', 'award', 'book', 'conference', 'event', 'magazine', 'misc', 'Organisation', 'theory']
politician ['country', 'discipline', 'election', 'person', 'politics', 'politician', 'writer']
enzyme ['algorithm', 'chemical', 'enzyme', 'field', 'product', 'protein']
astronomer ['astronomer', 'researcher', 'scientist', 'task', 'university']


In [ ]:
# for group in groups_emb:
#     ## find distance to centroid
#     label = ws.find_label_for_cluster_emb(word2vec, group)
#     print(label, group)

event ['award', 'conference', 'event', 'location']
country ['band', 'country', 'discipline', 'election', 'field', 'genre', 'misc', 'instrument', 'Organisation', 'politics', 'researcher', 'task', 'university']
person ['astronomer', 'artist', 'person', 'politician', 'scientist']
book ['journal', 'album', 'book', 'magazine', 'poem', 'song', 'theory', 'writer']
product ['algorithm', 'chemical', 'enzyme', 'metrics', 'product', 'java', 'protein']


In [ ]:
# for group in groups_emb:
#     # find most similar word
#     centroid = ws.find_vector_for_cluster(word2vec, group)
#     most_sim = word2vec.most_similar(centroid)[0]
#     print(most_sim)

('event', 0.687100887298584)
('Lisa_Vorderbrueggen_covers', 0.5613850355148315)
('scientist', 0.7774226069450378)
('poem', 0.7118237018585205)
('enzyme', 0.7614861726760864)


In [ ]:
entity2cluster_dist = ws.create_entity2cluster_dict(pred_dist)
entity2cluster_emb = ws.create_entity2cluster_dict(pred_emb)

# cluster2label_dist = ws.create_cluster2label_dict(word2vec, pred_dist)
# cluster2label_emb = ws.create_cluster2label_dict(word2vec, pred_emb)


## Mapping entities

In [ ]:
## load csv
df = pd.read_csv('../data/category_mappings.csv')
## get entity names
entities = df["entity_name"]

In [ ]:
def find_problematic_words(word2vec_model, entities):
    PROBLEMATIC_WORDS = []
    for entity in entities:
        try:
            word2vec_model[entity]
        except:
            PROBLEMATIC_WORDS.append(entity)
    return PROBLEMATIC_WORDS

In [ ]:
## find the problematic words
problem_words = find_problematic_words(word2vec_model=word2vec, entities=entities)
problem2work = {"musicalartist": "artist", "organisation": "Organisation", "politicalparty": "politics", "academicjournal": "journal", "chemicalcompound": "chemical", "chemicalelement": "chemical", "astronomicalobject": "astronomer", "musicgenre": "genre", "literarygenre": "genre", "programlang": "java", "musicalinstrument": "instrument"  }

In [ ]:
## map the problematic entities to working ones
working = entities.map(lambda entity: problem2work[entity] if entity in problem2work else entity)

In [ ]:
## map the words to their cluster(word) label
mapped_words_dist = working.map(lambda entity: cluster2label_dist[entity2cluster_dist[entity]])
# mapped_words_emb = working.map(lambda entity: cluster2label_emb[entity2cluster_emb[entity]])

In [ ]:
## add the new columns to the dataframe
df["word_dist"] = mapped_words_dist
# df["word_emb"] = mapped_words_emb

In [ ]:
## write to csv
df.to_csv('../data/category_mappings.csv', index=False)

In [ ]:
file = pd.read_csv("../data/category_mappings.csv", index_col=0)

In [ ]:
file["ours"]["politicalparty"]

'organisation'

In [ ]:
df_to_use = file["ours"]